In [3]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as ss
from scipy.stats import skewnorm

from matplotlib.cm import ScalarMappable
from matplotlib.lines import Line2D
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from textwrap import wrap

from mdutils.mdutils import MdUtils
from mdutils import Html
import networkx as nx
import random
import pyodbc
import math
import sys

from abc import ABC, abstractmethod

from networkx.drawing.nx_agraph import write_dot, graphviz_layout

import warnings

warnings.filterwarnings('ignore')

In [4]:
def Dice(x,y,**kwargs):
    z = kwargs.get('drop_num')
    high = kwargs.get('drop_high')
    low = kwargs.get('drop_low')
    dice = y+1
    bottom = kwargs.get('bottom',1)
    roll_rng = range(bottom,dice)
    roll = [*roll_rng]
    if bool(z):
        roll.remove(z)
    # print(roll)
    result = []
    for i in range(1,x+1):
        res = np.random.choice(roll)
        result.append(res)
    # print(result)
    if bool(high):
        result.sort(reverse=False)
        result.pop()
    # print(result)
    if bool(low):
        result.sort(reverse=True)
        result.pop()
    # print(result)
    Sum = sum(result)
    return Sum


def setall(d, keys, value):
    for k in keys:
        d[k] = value

In [ ]:
class Hex:
    # Initializing constructor
    def __init__(self, **kwargs):
        self.Terrain = kwargs.get('Ter')
        self.Climate = kwargs.get('Clim')
        self.Location = kwargs.get('Loc')
        self.Num_Features = kwargs.get('NF',Dice(1,6))
        self.Num_Lairs = kwargs.get('NL',Dice(1,6))
 
    # def isMammal(self):
    #     if self.mammals:
    #         print("It is a mammal.")
 
    # def isIntelligent(self):
    #     if self.intelligent:
    #         print("It is an intelligent creature.")

    # def name(self,f,l,m):
    #     self.first = f
    #     self.last = l
    #     self.mid = m
    #     self.full = (self.first,self.mid,self.last)

class Lair:
    def __init__(self, **kwargs):
        self.Number = kwargs.get('Num')


In [63]:
Biome_Codes = {
(0,0.125):  ['Polar/Alvar Desert', 'Glacier', 'Barren Rock','Frozen Earth','Ice Sheet'],
(0,0.25):   ['Polar/Alvar Desert', 'Glacier', 'Barren Rock','Frozen Earth','Ice Sheet'],
(0,0.5):    ['Polar/Alvar Desert', 'Glacier', 'Barren Rock','Frozen Earth','Ice Sheet'],
(1,0.125):  ['Rain Tundra',],
(1,0.25):   ['Wet Tundra',],
(1,0.5):    ['Moist Tundra',],
(1,1):      ['Dry Tundra','Mammoth Steppe'],
(2,0.125):  ['Boreal Rain Forest',],
(2,0.25):   ['Wet Boreal Forest',],
(2,0.5):    ['Moist Boreal Forest',],
(2,1):      ['Dry Scrub',],
(2,2):      ['Desert',],
(3,0.125):  ['Rain Forest',],
(3,0.25):   ['Wet Forest',],
(3,0.5):    ['Moist Forest',],
(3,1):      ['Steppe',],
(3,2):      ['Desert Scrub',],
(3,4):      ['Desert',],
(4,0.125):  ['Rain Forest',],
(4,0.25):   ['Wet Forest',],
(4,0.5):    ['Moist Forest',],
(4,1):      ['Dry Forest',],
(4,2):      ['Thorn Steppe',],
(4,4):      ['Desert Scrub',],
(4,8):      ['Desert',],
(5,0.125):  ['Rain Forest',],
(5,0.25):   ['Wet Forest',],
(5,0.5):    ['Moist Forest',],
(5,1):      ['Dry Forest',],
(5,2):      ['Very Dry Forest',],
(5,4):      ['Thorn Woodland',],
(5,8):      ['Desert Scrub',],
(5,16):     ['Desert',    ],
}

humidity_code = {
    0.125:'Super-Humid',
    0.25:'Per-Humid',
    0.5:'Humid',
    1:'Sub-Humid',
    2:'Semi-Arid',
    4:'Arid',
    8:'Perarid',
    16:'Super-Arid',
}

Precipitation_Probability = {
    'Super-Humid':90,
    'Per-Humid':80,
    'Humid':65,
    'Sub-Humid':32,
    'Semi-Arid':16,
    'Arid':8,
    'Perarid':4,
    'Super-Arid':2,
}

Altitude_belts = [
    ("Alvar",0),
    ("Alpine",1),
    ("Subalpine",2),
    ("Montant",3),
    ("Lower Montane",4),
   ( "Premontane",4),
   ( "Lowland",5)
]
Latitude_regions = [
    ("Polar",0),
    ("Subpolar",1),
    ("Boreal",2),
    ("Cold Temperate",3),
    ("Warm Temperate",4),
    ("Subtropical",4),
    ("Tropical",5)
]

temp_belts = {
    0:'Frozen',
    1:'Cold',
    2:'Cool',
    3:'Mild',
    4:'Warm',
    5:'Hot',
}

Avg_Temp = {
    0:(0,40), #(Avg , Std. Dev)
    1:(15,30),
    2:(45,25),
    3:(60,20),
    4:(80,15),
    5:(100,10),
}

Ocean_Dist_Bands = {
    0:'Ocean', ## This hex is ocean water
    1:'Small Island', ## This hex is surrounded on all sides by Ocean hexes
    2:'Beach', ## At least one side of this hex borders the ocean, and at least one side does not
    3:'Coastal', ## At least one side of this of this hex borders a beach hex, but it does not border any ocean hexes
    4:'Inland', ## At least one side of this hex borders a coastal hex, but it does not border any beach or ocean hexes
    5:'Mainland', ##This hex only borders Mainland or Inland hexes; it cannot border any other hex-type from this list
}

Ocean_Effect = {
    0:-20, ### Effect of distance from ocean on Temperature Std. Dev.
    1:-15,
    2:-10,
    3:-5,
    4:0,
    5:10,
}

class Biome:
    def __init__(self, **kwargs):
        self.Altitude = kwargs.get('Alt',random.choice(Altitude_belts))
        self.Latitude = kwargs.get('Alt',random.choice(Latitude_regions))
        self.biotemp = min(self.Altitude[1],self.Latitude[1])
        Evapotrans_ratios = [2**n for n in range(-3,self.biotemp)]
        self.Evap = kwargs.get('Evap',random.choice(Evapotrans_ratios))
        self.Humid = kwargs.get('Humid',humidity_code[self.Evap])
        self.code = (self.biotemp,self.Evap)
        self.Biome = kwargs.get('biome',Biome_Codes[self.code])
        self.Temp = kwargs.get('temp',temp_belts[self.biotemp])
        self.Ocean_Dist = kwargs.get('OD',)



In [64]:
B = Biome()
print(B.Altitude[0])
print(B.Latitude[0])
print(B.Temp)
print(B.Humid)
print(B.Biome)

Alvar
Cold Temperate
Frozen
Humid
Cold Desert


In [11]:
Soil = [
    "topsoil",
    "Volcanic Glass",
    "Sand",
    "Sandy Soil",
    "Clay",
    "Loam"
]
Rocks = [
    "Boulders",
    "Large rocks",
    "Small Rocks",
    "Pebbles",
    "None"
]

groundwater_type = [
    "freshwater",
    "salt water",
    "brackish"
]
underground_aquifer = [
    "yes",
    "no"
]

Unusual_terrain = [
    "Glacier",
    "Hot Springs",
    "Landlocked Sand Dunes",
    "Mudflats",
    "Petrified Forest",
    "Rare Rock Formations",
    "Salt Flat",
    "Volcano",
    "Craters",
    
]


In [35]:
Evapotrans_ratio = [2**n for n in range(-3,5)]
precip = [500*n for n in Evapotrans_ratio]
print(Evapotrans_ratio)
print(precip)
print(len(precip)*6)

[0.125, 0.25, 0.5, 1, 2, 4, 8, 16]
[62.5, 125.0, 250.0, 500, 1000, 2000, 4000, 8000]
48
